In [6]:
### Run this notebook in the qiime2 conda installation environoment and add the Biopython module 

import re,os, subprocess
import pandas as pd
import qiime2
import glob
from qiime2 import Metadata
from qiime2.plugins import feature_table
import qiime2.plugins.dada2.actions as dada2
from qiime2.plugins.vsearch.methods import uchime_denovo
from qiime2.plugins.feature_table.methods import filter_features
from qiime2.plugins.feature_classifier.methods import classify_sklearn

from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import itertools


def create_manifest(directory):
    files = [directory+'/'+file for file in os.listdir(directory) if file.endswith('.fastq.gz')]
    os.chdir(directory)
    with open('manifest.txt', 'w') as mani:
        mani.write('sample-id,absolute-filepath,direction\n')
        for file in files:
            sampleID =  re.match(re.compile(r'(\w+)_R\d\.fastq\.gz'), os.path.basename(file)).group(1)
            direction = re.match(re.compile(r'\w+_R(\d)\.fastq\.gz'), os.path.basename(file)).group(1)
            if direction == "1":
                direction = 'forward'
            else:
                direction = 'reverse'

            mani.write(sampleID+','+file+','+direction+'\n')
    mani.close()
    
def merge_complementary_asv(Fasta, CountsDf, Tag):
    """Creates a new fasta and a new asv_table that are dereplicated"""
    #CountsDf  = CountsDf.copy()
    FastaRec = list(SeqIO.parse(open(Fasta,'r'),'fasta'))
    pairCount = 0
    for rec1, rec2 in itertools.combinations(FastaRec, 2):
        Seq1 = str(rec1.seq.reverse_complement())
        if Seq1 == str(rec2.seq) or rec1.seq == rec2.seq:
            pairCount += 1
            CountsDf[rec1.id] = CountsDf[rec1.id] + CountsDf[rec2.id] ## arbitarily keep the first in the pair
            CountsDf.drop(columns=[rec2.id], inplace=True) ## after the complementary ASVs are merged, remove the second of them from the table.
    with open("asv_seqs.fasta", 'w') as out:
        NewFasta = [rec for rec in FastaRec if rec.id in CountsDf.columns]
        SeqIO.write(NewFasta, out, 'fasta')
    CountsDf.to_csv('final_asv_table.tsv', sep='\t', index_label='SampleID')                                                   
    print("{} ASVs were merged.".format(pairCount)) 
    print("Exporting final_asv_table.tsv!")
    print('Exporting '+Tag+'_drep_dna-sequences.fasta')
    out.close()
    return CountsDf

def parse_qiime2_taxonomy(TaxFile):
    Tax = pd.read_csv(TaxFile, sep='\t', index_col='Feature ID')
    TaxDf = pd.DataFrame()
    columns = ["domain", "phylum", "class", "order", 'family', 'genus', 'species', 'full', 'confidence']

    for index, row in Tax.iterrows():
        SplitTaxonomy = re.findall(r'__(\w+);*', row[0])
        while len(SplitTaxonomy) < 7:
            SplitTaxonomy.append('Unassigned')
        TaxDf = TaxDf.append(pd.DataFrame(dict(zip(columns, SplitTaxonomy+[row['Taxon'], row['Confidence']])), index=[index], columns=columns))

    TaxDf = TaxDf.assign(asv=['ASV_'+str(i) for i in range(1,TaxDf.shape[0]+1)])
    TaxDf = TaxDf.assign(ASVg=TaxDf[['asv','genus']].apply(lambda x:':'.join(x), axis=1))
    TaxDf.reset_index(inplace=True)
    TaxDf.rename(columns={'index':'asvid'}, inplace=True)
    return TaxDf


## Directory with the concatenated fastq files..
myDir = '/media/christos/ssd/work/Infants/publication/sequence_data'

### Root directory to be used for the project..
rootDir = ''

### all files exported from this notebook can be found here! 
exportDir = rootDir+'qiime2_dada2'


## Import files in qiime2

In [7]:
create_manifest(myDir)
manifesto = myDir+'/manifest.txt'

### Parse files to qiime2
demultiplexed_fastqs = qiime2.Artifact.import_data('SampleData[PairedEndSequencesWithQuality]', manifesto, view_type='PairedEndFastqManifestPhred33')

## Run DADA2

In [8]:
asv_table, repr_seqs, denoising_stasts  = dada2.denoise_paired(demultiplexed_seqs=demultiplexed_fastqs,
                                                               trunc_len_f=230, trunc_len_r=230,
                                                               max_ee=20.0, n_threads=3)

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada_paired.R /tmp/tmpw34n7fqm/forward /tmp/tmpw34n7fqm/reverse /tmp/tmpw34n7fqm/output.tsv.biom /tmp/tmpw34n7fqm/track.tsv /tmp/tmpw34n7fqm/filt_f /tmp/tmpw34n7fqm/filt_r 230 230 0 0 20.0 2 consensus 1.0 3 1000000



In [9]:
os.chdir(exportDir)
asv_df = asv_table.view(pd.DataFrame)
stats_df = denoising_stasts.view(Metadata).to_dataframe()
total_reads = asv_df.sum(axis=1).to_frame().rename(columns={0:"Total"})
stats_df.to_csv('ReadStats.csv') ## export read stats

## Remove chimeras with the vsearch uchime_denovo method

In [10]:
### check and remove chimeras - uses dada2 output 
chimeras, non_chimeras, stats = uchime_denovo(sequences=repr_seqs, table=asv_table)
#non_chimeras.save(tag+'_non_chimeras.qza')
non_chimeras_df = non_chimeras.view(Metadata).to_dataframe() ### convert fasta Artifatct to dataframe
asv_df = asv_table.view(pd.DataFrame).loc[:,non_chimeras_df.index] ## filter asv table for non-chimeric seqeuences
repr_seqs_df = repr_seqs.view(Metadata).to_dataframe().loc[non_chimeras_df.index,:]## filter fasta file for non-chimeric sequences
print('{} ASVs detected.'.format(asv_df.shape[1]))
## write fasta record with non-chimeric sequences (to be used downstream for reverse complement dereplication)
with open('preterm_nochimera_seqs.fasta', 'w') as out:
    for index, row in repr_seqs_df.iterrows():
        record = SeqRecord(Seq(row['Sequence']), id=index, description=index)
        SeqIO.write(record, out, 'fasta')
out.close() 

#### update the Readstats.csv 
stats = pd.read_csv('ReadStats.csv')
tmp = asv_df.sum(axis=1).to_frame().rename(columns={0:"After vsearch uchime"}).reset_index().rename(columns={'index':'sample-id'})
stats.merge(tmp, on='sample-id', how='inner').to_csv('ReadStats.csv', index=False)


Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: vsearch --uchime_denovo /tmp/tmpnl40uzqd --uchimeout /tmp/q2-UchimeStatsFmt-xfj0asai --nonchimeras /tmp/q2-DNAFASTAFormat-tv6i66cc --chimeras /tmp/tmpjl8tdyde --dn 1.4 --mindiffs 3 --mindiv 0.8 --minh 0.28 --xn 8.0 --qmask none --xsize

287 ASVs detected.


## Dereplicate reverse complement ASVs
This section is the final processing step for the count table in the qiime2 pipeline. ***final_asv_table.tsv*** and ***asv_seqs.fasta*** exported files can be further used for downstream analysis.


In [11]:
fasta_file = 'preterm_nochimera_seqs.fasta'

## This function will dereplicate the table and the respective file for reverse-complementary sequences 
asv_df = merge_complementary_asv(fasta_file, asv_df, Tag='preterm')

#### update the Readstats.csv 
stats = pd.read_csv('ReadStats.csv')
tmp = asv_df.sum(axis=1).to_frame().rename(columns={0:"After rev_com_derep"}).reset_index().rename(columns={'index':'sample-id'})
stats.merge(tmp, on='sample-id', how='inner').to_csv('ReadStats.csv', index=False)

query = qiime2.Artifact.import_data('FeatureData[Sequence]',exportDir+'/asv_seqs.fasta')
query.save(exportDir+'/asv_seqs.qza')

95 ASVs were merged.
Exporting final_asv_table.tsv!
Exporting preterm_drep_dna-sequences.fasta


'/media/christos/ssd/work/Infants/publication/qiime2_dada2/asv_seqs.qza'

## Taxonomic classification 
A classifier is trained for the region amplified by the primer pair 341F-785R using SILVA SSU 16S release 132 - check *trainFeatureClassifier.py* script). Keep in mind that ASV sequences derived from both DNA strands. Therefore, assign taxonomies for all ASVs for both  DNA strands. Exports ***taxonomy_table.tsv*** and ***final_asv_seqs.fasta*** . (Train the classifier and make the classification on a system with > 20GB of RAM. Then extract taxonomy folders and rename accordingly the taxonomy files to *taxonomy_f.tsv* and *taxonomy_r.tsv* respectively ) 


##### Train the classifier
qiime feature-classifier fit-classifier-naive-bayes --i-reference-reads ref-reads.qza --i-reference-taxonomy ref-taxonomy.qza --o-classifier 16S-silva-341f-785r-classifier.qza --p-classify--chunk-size 20000

##### Assign taxonomies forward
qiime feature-classifier classify-sklearn --i-classifier 16S-silva-341f-785r-classifier.qza --i-reads asv_seqs.qza --o-classification taxonomy_f.qza

##### Assign taxonomies reverse complement
qiime feature-classifier classify-sklearn --p-read-orientation reverse-complement --i-classifier 16S-silva-341f-785r-classifier.qza --i-reads asv_seqs.qza --o-classification taxonomy_r.qza



In [13]:
## Merge forward and reverse taxonomic assignments 
os.chdir(exportDir)

tax1 = 'taxonomy_f.tsv'
tax2 = 'taxonomy_r.tsv'

tax1_df  = parse_qiime2_taxonomy(tax1)
tax1_df['strand'] = ['f' for  i in tax1_df.index]
tax2_df = parse_qiime2_taxonomy(tax2)
tax2_df['strand'] =['r' for i in tax2_df.index]

tax_df = tax1_df.append(tax2_df, ignore_index=True)
tax_df['badscore'] = tax_df.apply(lambda row: sum(row[0:tax_df.shape[1]]=="Unassigned"), axis=1)
tax_df = tax_df.sort_values(['asvid','badscore'], ascending=True)
tax_df.drop_duplicates("asvid", inplace=True)
otuIDs = tax_df['asvid']

"""Check the orientation and then reverse complement the sequences in the FASTA. (should be asv_seqs.fasta)"""
RC_otus = tax_df.loc[tax_df['strand'] == 'r']['asvid'].tolist()
#tax_df.set_index('asvid',inplace=True)
with open("asv_seqs.fasta",'r') as old, open('final_asv_seqs.fasta','w') as new:
    for record in SeqIO.parse(old,'fasta'):
        if record.id in RC_otus:
            record.seq = record.seq.reverse_complement()
        SeqIO.write(record, new, 'fasta')
old.close(), new.close()
tax_df.drop(columns=['strand'], inplace=True)
tax_df.to_csv('taxonomy_table.tsv', index=None, sep='\t')

#qtax_df.to_csv('/media/christos/ssd/work/Infants/qiime2/final-fulltaxonomy-table.tsv',index=None, sep='\t')
